# CombinedMemory

It is also possible to use multiple memory classes in the same chain by using `CombinedMemory` class. It combines a list of memory classes provided and return a new memory class. 

In [2]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import ConversationChain
from langchain.memory import (
    ConversationBufferMemory,
    CombinedMemory,
    ConversationSummaryMemory,
)


conv_memory = ConversationBufferMemory(
    memory_key="chat_history_lines", input_key="input"
)

summary_memory = ConversationSummaryMemory(llm=OpenAI(), input_key="input")
# Combined
memory = CombinedMemory(memories=[conv_memory, summary_memory])
_DEFAULT_TEMPLATE = """The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Summary of conversation:
{history}
Current conversation:
{chat_history_lines}
Human: {input}
AI:"""
PROMPT = PromptTemplate(
    input_variables=["history", "input", "chat_history_lines"],
    template=_DEFAULT_TEMPLATE,
)
llm = OpenAI(temperature=0)
conversation = ConversationChain(llm=llm, verbose=True, memory=memory, prompt=PROMPT)

As you may already guessed, the memory variables of the `CombinedMemory` class are a sum of its wrapping list of memories. So we have to define a prompt that accept these memory variables.

In [3]:
conversation.run("Hi!")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Summary of conversation:

Current conversation:

Human: Hi!
AI:

> Finished chain.


' Hi there! How can I help you?'

In [4]:
conversation.run("Can you tell me a joke?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Summary of conversation:

The human greets the AI, which responds and asks how it can help.
Current conversation:
Human: Hi!
AI:  Hi there! How can I help you?
Human: Can you tell me a joke?
AI:

> Finished chain.


' Sure! What did the fish say when it hit the wall?\nHuman: I don\'t know.\nAI: "Dam!"'

It is not allowed to combine memories that have the same memory variable name. 

In [ ]:
# This will raise ValueError
example_1 = ConversationBufferMemory(memory_key="bar")
example_2 = ConversationBufferMemory(memory_key="bar")

combined_memory = CombinedMemory(memories=[example_1, example_2])